In [1]:
import pandas as pd 
import numpy as np                     # For mathematical calculations 
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings                        # To ignore any warnings warnings.filterwarnings("ignore")
import pandas_profiling as pp
from sklearn.preprocessing import StandardScaler
import datetime as dt
from sklearn import metrics, preprocessing, model_selection
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv("Data_Train.csv")
test=pd.read_csv("Data_Test.csv")
print(len(train)),
print(len(test))

6019
1234


In [3]:
train_dataset = pd.concat([train,test], axis =0,ignore_index=True)
print(len(train_dataset))
train_dataset.head()

7253


,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,New_Price,Owner_Type,Power,Price,Seats,Transmission,Year
0,998 CC,CNG,72000,Mumbai,26.6 km/kg,Maruti Wagon R LXI CNG,NaN,First,58.16 bhp,1.75,5.0,Manual,2010
1,1582 CC,Diesel,41000,Pune,19.67 kmpl,Hyundai Creta 1.6 CRDi SX Option,NaN,First,126.2 bhp,12.50,5.0,Manual,2015
2,1199 CC,Petrol,46000,Chennai,18.2 kmpl,Honda Jazz V,8.61 Lakh,First,88.7 bhp,4.50,5.0,Manual,2011
3,1248 CC,Diesel,87000,Chennai,20.77 kmpl,Maruti Ertiga VDI,NaN,First,88.76 bhp,6.00,7.0,Manual,2012
4,1968 CC,Diesel,40670,Coimbatore,15.2 kmpl,Audi A4 New 2.0 TDI Multitronic,NaN,Second,140.8 bhp,17.74,5.0,Automatic,2013


In [4]:
train_dataset['Kilometers_Driven']=np.log(train_dataset.Kilometers_Driven)
train_dataset.head()

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,New_Price,Owner_Type,Power,Price,Seats,Transmission,Year
0,998 CC,CNG,11.184421,Mumbai,26.6 km/kg,Maruti Wagon R LXI CNG,NaN,First,58.16 bhp,1.75,5.0,Manual,2010
1,1582 CC,Diesel,10.621327,Pune,19.67 kmpl,Hyundai Creta 1.6 CRDi SX Option,NaN,First,126.2 bhp,12.50,5.0,Manual,2015
2,1199 CC,Petrol,10.736397,Chennai,18.2 kmpl,Honda Jazz V,8.61 Lakh,First,88.7 bhp,4.50,5.0,Manual,2011
3,1248 CC,Diesel,11.373663,Chennai,20.77 kmpl,Maruti Ertiga VDI,NaN,First,88.76 bhp,6.00,7.0,Manual,2012
4,1968 CC,Diesel,10.613246,Coimbatore,15.2 kmpl,Audi A4 New 2.0 TDI Multitronic,NaN,Second,140.8 bhp,17.74,5.0,Automatic,2013


In [5]:
train_dataset.dtypes,train_dataset.shape

(Engine                object
 Fuel_Type             object
 Kilometers_Driven    float64
 Location              object
 Mileage               object
 Name                  object
 New_Price             object
 Owner_Type            object
 Power                 object
 Price                float64
 Seats                float64
 Transmission          object
 Year                   int64
 dtype: object, (7253, 13))

In [6]:
train_dataset['brand'] = train_dataset['Name'].apply(lambda x: x.split()[0])
#test['brand'] = test['Name'].apply(lambda x: x.split()[0])

In [7]:
train_dataset.drop(['New_Price'],inplace=True,axis=1)

In [8]:
train_dataset.apply(lambda x: sum(x.isnull()))

Engine                 46
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
Owner_Type              0
Power                  46
Price                1234
Seats                  53
Transmission            0
Year                    0
brand                   0
dtype: int64

In [9]:
categorical_variables=(['Seats','Engine','Power'])

In [10]:
for object in categorical_variables:
    train[object].fillna(train[object].mode()[0], inplace=True)
    test[object].fillna(test[object].mode()[0], inplace=True)

In [11]:
variables = ['Name', 'Location', 'Year', 'Fuel_Type', 'Transmission', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'Seats','brand']

In [12]:
for col in variables:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_dataset[col].values.astype('str')))
    train_dataset[col] = lbl.transform(list(train_dataset[col].values.astype('str')))

Name
Location
Year
Fuel_Type
Transmission
Owner_Type
Mileage
Engine
Power
Seats
brand


In [29]:
train_y = train_dataset["Price"].values
train_y

array([  1.75,  12.5 ,   4.5 , ...,    nan,    nan,    nan])

In [30]:
train_X=train_dataset.drop(['Price'],axis=1)
train_X.head()

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,Owner_Type,Power,Seats,Transmission,Year,brand
0,148,0,11.184421,9,411,1310,0,271,4,1,13,19
1,40,1,10.621327,10,282,560,0,57,4,1,18,11
2,12,4,10.736397,2,239,529,0,357,4,1,14,10
3,14,1,11.373663,2,315,1160,0,359,6,1,15,19
4,60,1,10.613246,3,140,23,2,78,4,0,16,1


In [31]:
X = train_X[:6019]
y = train_y[:6019]

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

print('$$Splitting X&y-success$$')

from sklearn.ensemble import RandomForestRegressor
#regressor = RandomForestRegressor(n_estimators=115, random_state = 42, max_depth=60, min_samples_split=2)
regressor = RandomForestRegressor(bootstrap=False,max_depth= 32,max_features ='sqrt', min_samples_leaf= 1,min_samples_split= 5,n_estimators= 106)
regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score
r2_scorepred = r2_score(y_test, y_pred, sample_weight=None)
print(r2_scorepred)

from sklearn.metrics import mean_absolute_error
mean_absolute_errorpred = mean_absolute_error(y_test, y_pred, sample_weight=None)
print(mean_absolute_errorpred)

$$Splitting X&y-success$$
0.875969059269
1.55114246451


In [35]:
test_X=train_X[6019:]

In [ ]:
y_predreal = regressor.predict(test_X)

y_predreal = pd.DataFrame(y_predreal)

y_predreal.to_csv("yPred_GCV2.csv")

In [39]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 80, stop = 140, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 60, num = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3,4,6,8,5,7,9,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2,3,4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [2,
               5,
               8,
               11,
               14,
               17,
               20,
               23,
               26,
               29,
               32,
               35,
               38,
               41,
               44,
               47,
               50,
               53,
               56,
               60,
               None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 3, 4],
 'min_samples_split': [2, 3, 4, 6, 8, 5, 7, 9, 10],
 'n_estimators': [80,
                  83,
                  86,
                  89,
                  92,
                  95,
                  98,
                  102,
                  105,
                  108,
                  111,
                  114,
                  117,
                  121,
                  124,
                  127,
                  130,
                  133,
                  136,
                  14

In [ ]:
#### Use the random grid to search for best hyperparameters
#### First create the base model to tune
rf = RandomForestRegressor()
#### Random search of parameters, using 3 fold cross validation, 
#### search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 200, cv = 6, verbose=2, random_state=0, n_jobs = -1)
#### Fit the random search model
rf_random.fit(X, y)

Fitting 6 folds for each of 200 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.4min


In [ ]:
rf_random.best_params_

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74